In [1]:
# import csv
# import os
# import random
# import boto3
# import botocore
# import tqdm
# from concurrent import futures
# import pandas as pd
# import re
# import sys

# #Rec: Take 5000 from the beginning 

# # Constants
# BUCKET_NAME = 'open-images-dataset'
# REGEX = r'(test|train|validation|challenge2018)/([a-fA-F0-9]*)'

# ###################################
# # Step 1: #From a large dataset, find all images with at least 2 cars and write their IDs to a file -> car_images.txt
# ###################################
# def find_images_with_multiple_cars(output_file="car_images.txt",subset_size=5000, seed=1610):
#     """
#     Find all images in the Open Images training set with at least 2 cars and
#     write their IDs to a file in the format 'train/$id'.
    
#     Args:
#         output_file: Path to the output file where image IDs will be saved
#     """
#     print("Downloading Open Images annotations if needed...")
    
#     # Paths to annotation files
#     class_labels_file = "class-descriptions-boxable.csv"
#     train_annotations_file = "train-annotations-bbox.csv"
    
#     # Download the class descriptions if needed
#     if not os.path.exists(class_labels_file):
#         print(f"Downloading {class_labels_file}...")
#         os.system(f"wget https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv")
        
        
#     # Download the training annotations if needed
#     if not os.path.exists(train_annotations_file):
#         print(f"Downloading {train_annotations_file}...")
#         os.system(f"wget https://storage.googleapis.com/openimages/2018_04/train/train-annotations-bbox.csv")
    
#     # Get the class ID for 'Car'
#     car_class_id = None
#     print("Finding car class ID...")
#     with open(class_labels_file, 'r') as f:
#         reader = csv.reader(f)
#         for row in reader:
#             if row[1].lower() == 'car':
#                 car_class_id = row[0]
#                 break
    
#     if not car_class_id:
#         print("Error: Could not find class ID for 'Car'")
#         return
    
#     print(f"Car class ID: {car_class_id}")
    
#     # Count car occurrences per image
#     print("Reading annotations and counting cars per image...")
#     car_counts = {}
    
#     # Read the annotations file in chunks to handle its large size
#     chunk_size = 1000000
#     for chunk in tqdm.tqdm(pd.read_csv(train_annotations_file, chunksize=chunk_size)):
#         # Filter to only car annotations
#         car_annotations = chunk[chunk['LabelName'] == car_class_id]
        
#         # Count cars per image
#         for image_id in car_annotations['ImageID']:
#             car_counts[image_id] = car_counts.get(image_id, 0) + 1
    
#     # Filter images with at least 2 cars
#     images_with_multiple_cars = [img_id for img_id, count in car_counts.items() if count >= 2]
    
#     print(f"Found {len(images_with_multiple_cars)} images with at least 2 cars")
    
#     # Randomly sample a subset with seed
#     if len(images_with_multiple_cars) > subset_size:
#         random.seed(seed)
#         images_with_multiple_cars = random.sample(images_with_multiple_cars, subset_size)
#         print(f"Randomly selected {subset_size} images with seed={seed}")

#     with open(output_file, 'w') as f:
#         for img_id in images_with_multiple_cars:
#             f.write(f"train/{img_id}\n")
    
#     print(f"Image IDs saved to {output_file}")
#     print(f"You can now use the downloader script with this file: python downloader.py {output_file}")
    
    

# ###################################
# # Step 2: From the car_images.txt file, download images to the data folder 
# ###################################
# def check_and_homogenize_one_image(image):
#   split, image_id = re.match(REGEX, image).groups()
#   yield split, image_id

# def check_and_homogenize_image_list(image_list):
#   for line_number, image in enumerate(image_list):
#     try:
#       yield from check_and_homogenize_one_image(image)
#     except (ValueError, AttributeError):
#       raise ValueError(
#           f'ERROR in line {line_number} of the image list. The following image '
#           f'string is not recognized: "{image}".')

# def read_image_list_file(image_list_file):
#   with open(image_list_file, 'r') as f:
#     for line in f:
#       yield line.strip().replace('.jpg', '')


# def download_one_image(bucket, split, image_id, download_folder):
#   try:
#     bucket.download_file(f'{split}/{image_id}.jpg',
#                          os.path.join(download_folder, f'{image_id}.jpg'))
#   except botocore.exceptions.ClientError as exception:
#     sys.exit(
#         f'ERROR when downloading image `{split}/{image_id}`: {str(exception)}')

# def download_all_images(image_list_file, download_folder="data", num_processes=14):
#     bucket = boto3.resource('s3', config=botocore.config.Config(signature_version=botocore.UNSIGNED)).Bucket(BUCKET_NAME)

#     if not os.path.exists(download_folder):
#         os.makedirs(download_folder)
#     try:
#         image_list = list(check_and_homogenize_image_list(read_image_list_file(image_list_file)))
#     except ValueError as exception:
#         sys.exit(exception)

#     progress_bar = tqdm.tqdm(total=len(image_list), desc='Downloading images', leave=True)
#     with futures.ThreadPoolExecutor(max_workers=num_processes) as executor:
#         all_futures = [
#             executor.submit(download_one_image, bucket, split, image_id, download_folder)
#             for (split, image_id) in image_list
#         ]
#         for future in futures.as_completed(all_futures):
#             future.result()
#             progress_bar.update(1)
#     progress_bar.close()
    
# ###################################
# # Step 3: Given data folder, extract car labels from Open Images dataset -> export to car_labels.csv
# ###################################
# def extract_car_labels_for_images(image_list_file, output_file="car_labels.csv"):
#     """
#     Extract only car labels for images specified in image_list_file from Open Images dataset.
    
#     Args:
#         image_list_file: File containing image IDs in format 'train/[IMAGE_ID]'
#         output_file: Path to save extracted labels """
#     print("Reading image IDs...")
#     # Read image IDs from the file
#     image_ids = []
#     with open(image_list_file, 'r') as f:
#         for line in f:
#             # Extract just the ID part from "train/ID" format
#             parts = line.strip().split('/')
#             if len(parts) == 2:
#                 image_ids.append(parts[1])

#     image_ids_set = set(image_ids)
#     print(f"Found {len(image_ids_set)} unique image IDs")

#     # Check if annotation files exist
#     class_desc_file = "class-descriptions-boxable.csv"  
#     bbox_file = "train-annotations-bbox.csv" #Tao sua o day 

#     if not os.path.exists(class_desc_file):
#         print(f"Please download the class descriptions file first:")
#         print("wget https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv")
#         return
    
#     if not os.path.exists(bbox_file):
#         print(f"Please download the bounding box annotations file first:")
#         print("wget https://storage.googleapis.com/openimages/v6/oidv6-train-annotations-bbox.csv")
#         return
    
#     # Load class descriptions
#     print("Loading class descriptions...")
#     class_descriptions = {}
#     car_class_ids = []
    
#     with open(class_desc_file, 'r') as f:
#         reader = csv.reader(f)
#         for row in reader:
#             class_descriptions[row[0]] = row[1]
#             # Identify car-related classes
#             if any(car_term in row[1].lower() for car_term in ['car', 'truck', 'taxi', 'bus']):
#                 car_class_ids.append(row[0])

#     print(f"Found {len(car_class_ids)} car-related classes")

#     # Process annotations in chunks to handle large file size
#     print("Extracting car labels for your images...")
#     extracted_labels = []
    
#     # Process the file in chunks to save memory
#     chunk_size = 1000000
#     total_labels_found = 0

#     for chunk in tqdm.tqdm(pd.read_csv(bbox_file, chunksize=chunk_size)):
#         # Filter to only include our image IDs AND car-related classes
#         # matching_annotations = chunk[(chunk['ImageID'].isin(image_ids_set)) & 
#         #                             (chunk['LabelName'].isin(car_class_ids))]
#         matching_annotations = chunk[(chunk['ImageID'].isin(image_ids_set)) & 
#                               (chunk['LabelName'].isin(car_class_ids))].copy()

        
#         if not matching_annotations.empty:
#             total_labels_found += len(matching_annotations)
#             # Add class name based on LabelName ID
#             matching_annotations['LabelName_Text'] = matching_annotations['LabelName'].apply(
#                 lambda x: class_descriptions.get(x, 'Unknown'))
#             extracted_labels.append(matching_annotations)

#     if extracted_labels:
#         # Combine all chunks
#         all_labels = pd.concat(extracted_labels)
        
#         # Save to CSV
#         all_labels.to_csv(output_file, index=False)
        
#         print(f"Found {total_labels_found} car annotations for {len(all_labels['ImageID'].unique())} images")
#         print(f"Labels saved to {output_file}")
        
#         # Show counts by class
#         class_counts = all_labels['LabelName_Text'].value_counts()
#         print("\nTop 10 car class counts:")
#         print(class_counts.head(10))
#     else:
#         print("No matching car annotations found for your images")
    

# ###################################
# # Run all steps in order
# ###################################
# #Set num_processes according to the number of cores in your machine: Download takes 23 mins
# if __name__ == "__main__":
#     find_images_with_multiple_cars(output_file="car_images.txt",subset_size=5000, seed=1610)
#     download_all_images("car_images.txt", download_folder="data", num_processes=14)
#     extract_car_labels_for_images("car_images.txt")
    
    
#     # Load data 
#     labels = pd.read_csv("car_labels.csv") 
#     labels = labels[['ImageID','XMin', 'XMax', 'YMin','YMax', 'LabelName_Text']] 
#     labels.to_csv("car_labels.csv", index=False)
    
    
    
    

#####################################

import csv
import os
import random
import boto3
import botocore
import tqdm
from concurrent import futures
import pandas as pd
import re
import sys
from collections import defaultdict

# Constants
BUCKET_NAME = 'open-images-dataset'
REGEX = r'(test|train|validation|challenge2018)/([a-fA-F0-9]*)'
TARGET_CLASSES = ['car', 'truck', 'bus', 'taxi']

###################################
# Step 1: Find images with target vehicle classes and write their IDs to a file
###################################
def find_balanced_vehicle_images(output_file="vehicle_images.txt", images_per_class=1000, seed=1610):
    """
    Find images in the Open Images training set containing the target vehicle classes
    and create a balanced dataset with equal representation of each class.
    
    Args:
        output_file: Path to the output file where image IDs will be saved
        images_per_class: Number of images to collect per target class
        seed: Random seed for reproducibility
    """
    print("Downloading Open Images annotations if needed...")
    
    # Paths to annotation files
    class_labels_file = "class-descriptions-boxable.csv"
    train_annotations_file = "train-annotations-bbox.csv"
    
    # Download the class descriptions if needed
    if not os.path.exists(class_labels_file):
        print(f"Downloading {class_labels_file}...")
        os.system(f"wget https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv")
    
    # Download the training annotations if needed
    if not os.path.exists(train_annotations_file):
        print(f"Downloading {train_annotations_file}...")
        os.system(f"wget https://storage.googleapis.com/openimages/2018_04/train/train-annotations-bbox.csv")
    
    # Get class IDs for target classes
    print("Finding class IDs for target vehicle classes...")
    class_id_map = {}
    with open(class_labels_file, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            class_name = row[1].lower()
            for target_class in TARGET_CLASSES:
                if target_class.lower() == class_name:
                    class_id_map[row[0]] = target_class
                    print(f"Found {target_class} class ID: {row[0]}")
    
    if not class_id_map:
        print("Error: Could not find class IDs for target vehicle classes")
        return
    
    # Create a reverse mapping for easy lookup
    target_class_ids = list(class_id_map.keys())
    
    # Track images by class
    class_to_images = defaultdict(set)
    images_with_multiple_classes = defaultdict(set)
    
    # Read the annotations file in chunks to handle its large size
    print("Reading annotations and categorizing images by vehicle class...")
    chunk_size = 1000000
    for chunk in tqdm.tqdm(pd.read_csv(train_annotations_file, chunksize=chunk_size)):
        # Filter to only target class annotations
        vehicle_annotations = chunk[chunk['LabelName'].isin(target_class_ids)]
        
        # Group by image and class
        for _, row in vehicle_annotations.iterrows():
            image_id = row['ImageID']
            class_id = row['LabelName']
            class_name = class_id_map[class_id]
            class_to_images[class_name].add(image_id)
            
            # Also keep track of which classes each image has
            images_with_multiple_classes[image_id].add(class_name)
    
    # Print statistics
    for class_name, images in class_to_images.items():
        print(f"Found {len(images)} images with {class_name}")
    
    # Select a balanced set of images for each class
    selected_images = set()
    random.seed(seed)
    
    for class_name in TARGET_CLASSES:
        available_images = list(class_to_images.get(class_name, set()))
        
        if not available_images:
            print(f"Warning: No images found for class '{class_name}'")
            continue
            
        # Prioritize images that have only this class first
        exclusive_images = [img_id for img_id in available_images 
                           if len(images_with_multiple_classes[img_id]) == 1]
        
        # If we don't have enough exclusive images, use images with multiple classes
        if len(exclusive_images) >= images_per_class:
            class_selection = random.sample(exclusive_images, images_per_class)
        else:
            # Take all exclusive images first
            class_selection = exclusive_images
            
            # Then sample from images with multiple classes
            multi_class_images = [img_id for img_id in available_images 
                                 if len(images_with_multiple_classes[img_id]) > 1]
            
            remaining_needed = images_per_class - len(class_selection)
            if multi_class_images and remaining_needed > 0:
                additional = random.sample(multi_class_images, 
                                          min(remaining_needed, len(multi_class_images)))
                class_selection.extend(additional)
        
        selected_images.update(class_selection)
        print(f"Selected {len(class_selection)} images for {class_name}")
    
    print(f"Total selected images: {len(selected_images)}")
    
    # Write selected image IDs to file
    with open(output_file, 'w') as f:
        for img_id in selected_images:
            f.write(f"train/{img_id}\n")
    
    print(f"Image IDs saved to {output_file}")
    return selected_images

###################################
# Step 2: From the vehicle_images.txt file, download images to the data folder 
###################################
def check_and_homogenize_one_image(image):
    split, image_id = re.match(REGEX, image).groups()
    yield split, image_id

def check_and_homogenize_image_list(image_list):
    for line_number, image in enumerate(image_list):
        try:
            yield from check_and_homogenize_one_image(image)
        except (ValueError, AttributeError):
            raise ValueError(
                f'ERROR in line {line_number} of the image list. The following image '
                f'string is not recognized: "{image}".')

def read_image_list_file(image_list_file):
    with open(image_list_file, 'r') as f:
        for line in f:
            yield line.strip().replace('.jpg', '')

def download_one_image(bucket, split, image_id, download_folder):
    try:
        bucket.download_file(f'{split}/{image_id}.jpg',
                            os.path.join(download_folder, f'{image_id}.jpg'))
    except botocore.exceptions.ClientError as exception:
        print(f'ERROR when downloading image `{split}/{image_id}`: {str(exception)}')
        return False
    return True

def download_all_images(image_list_file, download_folder="data", num_processes=14):
    bucket = boto3.resource('s3', config=botocore.config.Config(signature_version=botocore.UNSIGNED)).Bucket(BUCKET_NAME)

    if not os.path.exists(download_folder):
        os.makedirs(download_folder)
    try:
        image_list = list(check_and_homogenize_image_list(read_image_list_file(image_list_file)))
    except ValueError as exception:
        sys.exit(exception)

    progress_bar = tqdm.tqdm(total=len(image_list), desc='Downloading images', leave=True)
    successful_downloads = []
    
    with futures.ThreadPoolExecutor(max_workers=num_processes) as executor:
        all_futures = {
            executor.submit(download_one_image, bucket, split, image_id, download_folder): (split, image_id)
            for (split, image_id) in image_list
        }
        for future in futures.as_completed(all_futures):
            result = future.result()
            split, image_id = all_futures[future]
            if result:
                successful_downloads.append(f"{split}/{image_id}")
            progress_bar.update(1)
    
    progress_bar.close()
    
    # Write successful downloads to a new file
    success_file = image_list_file.replace(".txt", "_successful.txt")
    with open(success_file, 'w') as f:
        for image_path in successful_downloads:
            f.write(f"{image_path}\n")
    
    print(f"Successfully downloaded {len(successful_downloads)} of {len(image_list)} images")
    print(f"Successful downloads list saved to {success_file}")
    return success_file
    
###################################
# Step 3: Extract vehicle labels from Open Images dataset -> export to vehicle_labels.csv
###################################
def extract_vehicle_labels_for_images(image_list_file, output_file="vehicle_labels.csv"):
    """
    Extract labels for target vehicle classes from images specified in image_list_file.
    
    Args:
        image_list_file: File containing image IDs in format 'train/[IMAGE_ID]'
        output_file: Path to save extracted labels 
    """
    print("Reading image IDs...")
    # Read image IDs from the file
    image_ids = []
    with open(image_list_file, 'r') as f:
        for line in f:
            # Extract just the ID part from "train/ID" format
            parts = line.strip().split('/')
            if len(parts) == 2:
                image_ids.append(parts[1])

    image_ids_set = set(image_ids)
    print(f"Found {len(image_ids_set)} unique image IDs")

    # Check if annotation files exist
    class_desc_file = "class-descriptions-boxable.csv"  
    bbox_file = "train-annotations-bbox.csv"

    if not os.path.exists(class_desc_file):
        print(f"Please download the class descriptions file first:")
        print("wget https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv")
        return
    
    if not os.path.exists(bbox_file):
        print(f"Please download the bounding box annotations file first:")
        print("wget https://storage.googleapis.com/openimages/2018_04/train/train-annotations-bbox.csv")
        return
    
    # Load class descriptions
    print("Loading class descriptions...")
    class_descriptions = {}
    vehicle_class_ids = []
    
    with open(class_desc_file, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            class_descriptions[row[0]] = row[1]
            # Filter for target vehicle classes
            if row[1] in TARGET_CLASSES or row[1].lower() in [c.lower() for c in TARGET_CLASSES]:
                vehicle_class_ids.append(row[0])

    print(f"Found {len(vehicle_class_ids)} target vehicle class IDs")

    # Process annotations in chunks to handle large file size
    print("Extracting vehicle labels for your images...")
    extracted_labels = []
    
    # Process the file in chunks to save memory
    chunk_size = 1000000
    total_labels_found = 0

    for chunk in tqdm.tqdm(pd.read_csv(bbox_file, chunksize=chunk_size)):
        # Filter to only include our image IDs AND target vehicle classes
        matching_annotations = chunk[(chunk['ImageID'].isin(image_ids_set)) & 
                                     (chunk['LabelName'].isin(vehicle_class_ids))].copy()
        
        if not matching_annotations.empty:
            total_labels_found += len(matching_annotations)
            # Add class name based on LabelName ID
            matching_annotations['LabelName_Text'] = matching_annotations['LabelName'].apply(
                lambda x: class_descriptions.get(x, 'Unknown'))
            extracted_labels.append(matching_annotations)

    if extracted_labels:
        # Combine all chunks
        all_labels = pd.concat(extracted_labels)
        
        # Check class balance
        class_counts = all_labels['LabelName_Text'].value_counts()
        print("\nClass distribution in the dataset:")
        print(class_counts)
        
        # Calculate class weights for potential use in model training
        total_samples = len(all_labels)
        class_weights = {}
        for class_name, count in class_counts.items():
            class_weights[class_name] = total_samples / (len(class_counts) * count)
        
        print("\nClass weights (for balanced training):")
        for class_name, weight in class_weights.items():
            print(f"{class_name}: {weight:.4f}")
        
        # Save to CSV
        all_labels.to_csv(output_file, index=False)
        
        print(f"\nFound {total_labels_found} vehicle annotations for {len(all_labels['ImageID'].unique())} images")
        print(f"Labels saved to {output_file}")
        
        # Return class counts for potential further processing
        return class_counts
    else:
        print("No matching vehicle annotations found for your images")
        return None

###################################
# Run all steps in order
###################################
if __name__ == "__main__":
    # Step 1: Find balanced set of images containing target classes
    selected_images = find_balanced_vehicle_images(output_file="vehicle_images.txt", images_per_class=1000, seed=1610)
    
    # Step 2: Download the selected images
    successful_downloads_file = download_all_images("vehicle_images.txt", download_folder="data", num_processes=14)
    
    # Step 3: Extract labels for the downloaded images
    class_counts = extract_vehicle_labels_for_images(successful_downloads_file, output_file="vehicle_labels.csv")
    
    # Load and prepare final data
    if os.path.exists("vehicle_labels.csv"):
        labels = pd.read_csv("vehicle_labels.csv") 
        labels = labels[['ImageID', 'XMin', 'XMax', 'YMin', 'YMax', 'LabelName_Text']] 
        labels.to_csv("vehicle_labels.csv", index=False)
        
        # Save basic dataset statistics
        stats = {
            "total_images": len(labels['ImageID'].unique()),
            "total_annotations": len(labels),
            "class_distribution": labels['LabelName_Text'].value_counts().to_dict()
        }
        
        pd.DataFrame([stats]).to_json("dataset_stats.json", orient="records")
        print("Final dataset statistics saved to dataset_stats.json")

Finding class IDs for target vehicle classes...
Found bus class ID: /m/01bjv
Found truck class ID: /m/07r04
Found car class ID: /m/0k4j
Found taxi class ID: /m/0pg52
Reading annotations and categorizing images by vehicle class...


15it [00:48,  3.23s/it]


Found 89465 images with car
Found 1434 images with taxi
Found 7293 images with bus
Found 8078 images with truck
Selected 1000 images for car
Selected 1000 images for truck
Selected 1000 images for bus
Selected 1000 images for taxi
Total selected images: 4000
Image IDs saved to vehicle_images.txt


Successfully downloaded 4000 of 4000 images
Successful downloads list saved to vehicle_images_successful.txt
Reading image IDs...
Found 4000 unique image IDs
Loading class descriptions...
Found 4 target vehicle class IDs
Extracting vehicle labels for your images...


15it [00:22,  1.53s/it]



Class distribution in the dataset:
LabelName_Text
Car      5368
Taxi     2913
Bus      1735
Truck    1565
Name: count, dtype: int64

Class weights (for balanced training):
Car: 0.5394
Taxi: 0.9939
Bus: 1.6687
Truck: 1.8500

Found 11581 vehicle annotations for 4000 images
Labels saved to vehicle_labels.csv
Final dataset statistics saved to dataset_stats.json


chuyển đổi nhãn từ file vehicle_labels.csv (định dạng Open Images: ImageID, LabelName_Text, XMin, XMax, YMin, YMax) sang định dạng YOLO (class_id x_center y_center width height).

In [17]:
import pandas as pd
import os

# Đường dẫn
data_dir = "data"
labels_csv = "vehicle_labels.csv"
labels_dir = "/Users/admin/Car-detection-serving-model/open_images_project/labels"
os.makedirs(labels_dir, exist_ok=True)

# Ánh xạ lớp
class_mapping = {
    'Car': 0,
    'Taxi': 1,
    'Truck': 2,
    'Bus': 3
}

# Đọc nhãn
labels = pd.read_csv(labels_csv)

# Chuyển đổi sang định dạng YOLO
for image_id in labels['ImageID'].unique():
    image_labels = labels[labels['ImageID'] == image_id]
    output_file = os.path.join(labels_dir, f"{image_id}.txt")
    with open(output_file, 'w') as f:
        for _, row in image_labels.iterrows():
            class_name = row['LabelName_Text']
            if class_name in class_mapping:
                class_id = class_mapping[class_name]
                x_min = row['XMin']
                x_max = row['XMax']
                y_min = row['YMin']
                y_max = row['YMax']
                x_center = (x_min + x_max) / 2
                y_center = (y_min + y_max) / 2
                width = x_max - x_min
                height = y_max - y_min
                f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

In [8]:
import pandas as pd
import os
import random
import shutil
import glob

# Xóa thư mục cũ
os.system("rm -rf /Users/admin/Car-detection-serving-model/open_images_project/labels")
os.system("rm -rf /Users/admin/Car-detection-serving-model/open_images_project/dataset")
os.system("rm -rf /Users/admin/runs/detect/train_large")  # Đảm bảo xóa lần huấn luyện trước

# Đường dẫn
data_dir = "data"
labels_csv = "vehicle_labels.csv"
labels_dir = "/Users/admin/Car-detection-serving-model/open_images_project/labels"
dataset_dir = "/Users/admin/Car-detection-serving-model/open_images_project/dataset"
os.makedirs(labels_dir, exist_ok=True)
os.makedirs(f"{dataset_dir}/images/train", exist_ok=True)
os.makedirs(f"{dataset_dir}/images/val", exist_ok=True)
os.makedirs(f"{dataset_dir}/labels/train", exist_ok=True)
os.makedirs(f"{dataset_dir}/labels/val", exist_ok=True)

# Ánh xạ lớp
class_mapping = {
    'Car': 0,
    'Taxi': 1,
    'Truck': 2,
    'Bus': 3
}

# Đọc nhãn
labels = pd.read_csv(labels_csv)

# Xóa duplicate labels
labels = labels.drop_duplicates(subset=['ImageID', 'LabelName_Text', 'XMin', 'YMin', 'XMax', 'YMax'])

# Chuyển đổi sang định dạng YOLO
for image_id in labels['ImageID'].unique():
    image_labels = labels[labels['ImageID'] == image_id]
    output_file = os.path.join(labels_dir, f"{image_id}.txt")
    with open(output_file, 'w') as f:
        for _, row in image_labels.iterrows():
            class_name = row['LabelName_Text']
            if class_name in class_mapping:
                class_id = class_mapping[class_name]
                x_min = row['XMin']
                x_max = row['XMax']
                y_min = row['YMin']
                y_max = row['YMax']
                x_center = (x_min + x_max) / 2
                y_center = (y_min + y_max) / 2
                width = x_max - x_min
                height = y_max - y_min
                f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

# Lấy danh sách ảnh có nhãn
images_with_labels = []
for img in os.listdir(data_dir):
    if not img.endswith(".jpg"):
        continue
    lbl = os.path.join(labels_dir, img.replace(".jpg", ".txt"))
    if os.path.exists(lbl) and os.path.getsize(lbl) > 0:  # Chỉ lấy ảnh có nhãn
        images_with_labels.append(img)

random.seed(1610)

# Chia train/val (90% train, 10% val)
train_imgs = random.sample(images_with_labels, int(0.9 * len(images_with_labels)))
val_imgs = [img for img in images_with_labels if img not in train_imgs]

# Sao chép ảnh và nhãn vào thư mục train
for img in train_imgs:
    shutil.copy(os.path.join(data_dir, img), f"{dataset_dir}/images/train/{img}")
    lbl = img.replace(".jpg", ".txt")
    shutil.copy(os.path.join(labels_dir, lbl), f"{dataset_dir}/labels/train/{lbl}")

# Sao chép ảnh và nhãn vào thư mục val
for img in val_imgs:
    shutil.copy(os.path.join(data_dir, img), f"{dataset_dir}/images/val/{img}")
    lbl = img.replace(".jpg", ".txt")
    shutil.copy(os.path.join(labels_dir, lbl), f"{dataset_dir}/labels/val/{lbl}")

# Kiểm tra phân bố lớp
class_counts = {i: 0 for i in range(len(class_mapping))}
invalid_files = []
for label_file in glob.glob(f"{dataset_dir}/labels/*/*.txt"):
    with open(label_file, "r") as f:
        for line in f:
            if line.strip():
                class_id = int(line.split()[0])
                if class_id in class_counts:
                    class_counts[class_id] += 1
                else:
                    invalid_files.append((label_file, class_id))

class_names = list(class_mapping.keys())
print("Phân bố lớp trong dataset lớn sau khi làm sạch:")
print("Train:")
for i in range(len(class_names)):
    print(f"{class_names[i]}: {class_counts[i]} instances")
print(f"Số ảnh train: {len(train_imgs)}")
print(f"Số ảnh val: {len(val_imgs)}")

if invalid_files:
    print("\nCác file chứa class_id không hợp lệ:")
    for file, class_id in invalid_files:
        print(f"File: {file}, class_id: {class_id}")
else:
    print("\nKhông tìm thấy class_id không hợp lệ.")

Phân bố lớp trong dataset lớn sau khi làm sạch:
Train:
Car: 5368 instances
Taxi: 2912 instances
Truck: 1565 instances
Bus: 1735 instances
Số ảnh train: 3600
Số ảnh val: 400

Không tìm thấy class_id không hợp lệ.


In [ ]:
import os

dataset_dir = "/Users/admin/Car-detection-serving-model/open_images_project/dataset"
data_yaml_content = f"""train: {dataset_dir}/images/train
val: {dataset_dir}/images/val
nc: 4
names: ['Car', 'Taxi', 'Truck', 'Bus']
# Data augmentation
hsv_h: 0.015  # Hue
hsv_s: 0.7    # Saturation
hsv_v: 0.4    # Value
degrees: 15.0  # Rotation
translate: 0.1  # Translation
scale: 0.5     # Scaling
shear: 0.0     # Shear
flipud: 0.5    # Flip up-down
fliplr: 0.5    # Flip left-right
mosaic: 1.0    # Mosaic
mixup: 0.0     # Mixup
"""
with open(f"{dataset_dir}/data.yaml", "w") as f:
    f.write(data_yaml_content)
print("Đã tạo data.yaml với 4 lớp và augmentation")

Đã tạo data.yaml với 4 lớp và augmentation


Huấn luyện lần đầu với mô hình pretrained

In [10]:
!pip install ultralytics
from ultralytics import YOLO

# Tải mô hình pretrained
model = YOLO("yolo11n.pt")

# Huấn luyện
results = model.train(
    data="/Users/admin/Car-detection-serving-model/open_images_project/dataset/data.yaml",
    epochs=30,  # 30 epochs
    imgsz=416,  # Tăng lên 416
    batch=16,   # Phù hợp CPU
    device="cpu",  # Chạy trên CPU
    name="train_large_new"
)


[notice] A new release of pip is available: 23.1.2 -> 25.1
[notice] To update, run: pip install --upgrade pip
New https://pypi.org/project/ultralytics/8.3.119 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.115 🚀 Python-3.11.4 torch-2.2.2 CPU (Intel Core(TM) i7-8559U 2.70GHz)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/Users/admin/Car-detection-serving-model/open_images_project/dataset/data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train_large_new, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None,

train: Scanning /Users/admin/Car-detection-serving-model/open_images_project/dataset/labels/train... 3600 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3600/3600 [00:02<00:00, 1251.45it/s]

train: New cache created: /Users/admin/Car-detection-serving-model/open_images_project/dataset/labels/train.cache


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 997.9±233.6 MB/s, size: 237.5 KB)


val: Scanning /Users/admin/Car-detection-serving-model/open_images_project/dataset/labels/val... 400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 400/400 [00:00<00:00, 1312.73it/s]

...

      30/30         0G     0.9466     0.8966      1.097         44        416: 100%|██████████| 225/225 [14:34<00:00,  3.89s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:33<00:00,  2.55s/it]

                   all        400       1203       0.79      0.594      0.711      0.515



30 epochs completed in 7.494 hours.
Optimizer stripped from /Users/admin/runs/detect/train_large_new/weights/last.pt, 5.4MB
Optimizer stripped from /Users/admin/runs/detect/train_large_new/weights/best.pt, 5.4MB

Validating /Users/admin/runs/detect/train_large_new/weights/best.pt...
Ultralytics 8.3.115 🚀 Python-3.11.4 torch-2.2.2 CPU (Intel Core(TM) i7-8559U 2.70GHz)
YOLO11n summary (fused): 100 layers, 2,582,932 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:29<00:00,  2.28s/it]


                   all        400       1203      0.789      0.594      0.711      0.515
                   Car        167        592      0.727      0.427       0.57      0.377
                  Taxi        104        298      0.806      0.529      0.685      0.435
                 Truck        103        136      0.829      0.713      0.809      0.618
                   Bus        101        177      0.793      0.706      0.779      0.632
Speed: 1.3ms preprocess, 51.2ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /Users/admin/runs/detect/train_large_new


Huấn luyện tiếp tục từ last.pt

In [ ]:
from ultralytics import YOLO

# Tải mô hình đã huấn luyện
model = YOLO("/Users/admin/runs/detect/train_large_new/weights/last.pt")

# Huấn luyện tiếp
results = model.train(
    data="/Users/admin/Car-detection-serving-model/open_images_project/dataset/data.yaml",
    epochs=10,  # Thêm 10 epochs
    imgsz=416,
    batch=16,
    device="cpu",
    name="train_large_new_extended"
)

New https://pypi.org/project/ultralytics/8.3.119 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.115 🚀 Python-3.11.4 torch-2.2.2 CPU (Intel Core(TM) i7-8559U 2.70GHz)
engine/trainer: task=detect, mode=train, model=/Users/admin/runs/detect/train_large_new/weights/last.pt, data=/Users/admin/Car-detection-serving-model/open_images_project/dataset/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train_large_new_extended, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, aug

train: Scanning /Users/admin/Car-detection-serving-model/open_images_project/dataset/labels/train.cache... 3600 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3600/3600 [00:00<?, ?it/s]

val: Fast image access ✅ (ping: 0.1±0.0 ms, read: 178.6±168.0 MB/s, size: 237.5 KB)



val: Scanning /Users/admin/Car-detection-serving-model/open_images_project/dataset/labels/val.cache... 400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 400/400 [00:00<?, ?it/s]


Plotting labels to /Users/admin/runs/detect/train_large_new_extended/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 416 train, 416 val
Using 0 dataloader workers
Logging results to /Users/admin/runs/detect/train_large_new_extended
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      10/10         0G     0.9698     0.9064        1.1         39        416: 100%|██████████| 225/225 [13:47<00:00,  3.68s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:30<00:00,  2.32s/it]

                   all        400       1203      0.758       0.63      0.705       0.51



10 epochs completed in 2.386 hours.
Optimizer stripped from /Users/admin/runs/detect/train_large_new_extended/weights/last.pt, 5.4MB
Optimizer stripped from /Users/admin/runs/detect/train_large_new_extended/weights/best.pt, 5.4MB

Validating /Users/admin/runs/detect/train_large_new_extended/weights/best.pt...
Ultralytics 8.3.115 🚀 Python-3.11.4 torch-2.2.2 CPU (Intel Core(TM) i7-8559U 2.70GHz)
YOLO11n summary (fused): 100 layers, 2,582,932 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:26<00:00,  2.05s/it]


                   all        400       1203      0.759       0.63      0.705      0.511
                   Car        167        592      0.692      0.478      0.563      0.373
                  Taxi        104        298       0.77      0.594      0.683      0.432
                 Truck        103        136      0.772      0.747      0.809      0.627
                   Bus        101        177      0.801      0.701      0.764       0.61
Speed: 1.3ms preprocess, 44.9ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /Users/admin/runs/detect/train_large_new_extended


Huấn luyện tiếp với learning rate thấp hơn

In [13]:
from ultralytics import YOLO

# Tải mô hình đã huấn luyện
model = YOLO("/Users/admin/runs/detect/train_large_new_extended/weights/last.pt")

# Huấn luyện tiếp với learning rate thấp hơn
results = model.train(
    data="/Users/admin/Car-detection-serving-model/open_images_project/dataset/data.yaml",
    epochs=10,  # Thêm 10 epochs
    imgsz=416,
    batch=16,
    device="cpu",
    lr0=0.0005,  # Giảm learning rate
    name="train_large_new_extended_2"
)

New https://pypi.org/project/ultralytics/8.3.119 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.115 🚀 Python-3.11.4 torch-2.2.2 CPU (Intel Core(TM) i7-8559U 2.70GHz)
engine/trainer: task=detect, mode=train, model=/Users/admin/runs/detect/train_large_new_extended/weights/last.pt, data=/Users/admin/Car-detection-serving-model/open_images_project/dataset/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train_large_new_extended_2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize

train: Scanning /Users/admin/Car-detection-serving-model/open_images_project/dataset/labels/train.cache... 3600 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3600/3600 [00:00<?, ?it/s]

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 387.4±160.6 MB/s, size: 237.5 KB)



val: Scanning /Users/admin/Car-detection-serving-model/open_images_project/dataset/labels/val.cache... 400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 400/400 [00:00<?, ?it/s]


Plotting labels to /Users/admin/runs/detect/train_large_new_extended_2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.0005' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 416 train, 416 val
Using 0 dataloader workers
Logging results to /Users/admin/runs/detect/train_large_new_extended_2
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      10/10         0G     0.9433     0.8428      1.084         39        416: 100%|██████████| 225/225 [14:57<00:00,  3.99s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:33<00:00,  2.54s/it]

                   all        400       1203       0.73      0.627      0.696      0.501



10 epochs completed in 2.475 hours.
Optimizer stripped from /Users/admin/runs/detect/train_large_new_extended_2/weights/last.pt, 5.4MB
Optimizer stripped from /Users/admin/runs/detect/train_large_new_extended_2/weights/best.pt, 5.4MB

Validating /Users/admin/runs/detect/train_large_new_extended_2/weights/best.pt...
Ultralytics 8.3.115 🚀 Python-3.11.4 torch-2.2.2 CPU (Intel Core(TM) i7-8559U 2.70GHz)
YOLO11n summary (fused): 100 layers, 2,582,932 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:30<00:00,  2.31s/it]


                   all        400       1203      0.728      0.631      0.696        0.5
                   Car        167        592      0.672      0.495       0.56      0.371
                  Taxi        104        298      0.741       0.57      0.671      0.425
                 Truck        103        136      0.761      0.743      0.797      0.598
                   Bus        101        177      0.738      0.717      0.755      0.607
Speed: 1.3ms preprocess, 51.8ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /Users/admin/runs/detect/train_large_new_extended_2


Cập nhật data.yaml với augmentation mới

In [14]:
dataset_dir = "/Users/admin/Car-detection-serving-model/open_images_project/dataset"
data_yaml_content = f"""train: {dataset_dir}/images/train
val: {dataset_dir}/images/val
nc: 4
names: ['Car', 'Taxi', 'Truck', 'Bus']
# Data augmentation
hsv_h: 0.015
hsv_s: 0.7
hsv_v: 0.4
degrees: 15.0  # Tăng từ 10.0 lên 15.0
translate: 0.1
scale: 0.5
shear: 0.0
flipud: 0.5
fliplr: 0.5
mosaic: 1.0
mixup: 0.0
"""
with open(f"{dataset_dir}/data.yaml", "w") as f:
    f.write(data_yaml_content)
print("Đã cập nhật data.yaml với degrees=15.0")

Đã cập nhật data.yaml với degrees=15.0


Huấn luyện lần cuối với cấu hình mới

In [15]:
from ultralytics import YOLO

# Tải mô hình đã huấn luyện
model = YOLO("/Users/admin/runs/detect/train_large_new_extended_2/weights/last.pt")

# Huấn luyện lần cuối
results = model.train(
    data="/Users/admin/Car-detection-serving-model/open_images_project/dataset/data.yaml",
    epochs=10,  # 10 epochs
    imgsz=640,  # Tăng từ 416 lên 640
    batch=16,
    device="cpu",
    optimizer="AdamW",  # Ép dùng AdamW
    lr0=0.0005,  # Cố định learning rate
    name="train_large_new_final"
)

New https://pypi.org/project/ultralytics/8.3.120 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.115 🚀 Python-3.11.4 torch-2.2.2 CPU (Intel Core(TM) i7-8559U 2.70GHz)
engine/trainer: task=detect, mode=train, model=/Users/admin/runs/detect/train_large_new_extended_2/weights/last.pt, data=/Users/admin/Car-detection-serving-model/open_images_project/dataset/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train_large_new_final, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=F

train: Scanning /Users/admin/Car-detection-serving-model/open_images_project/dataset/labels/train.cache... 3600 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3600/3600 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.4±0.2 ms, read: 322.8±192.0 MB/s, size: 237.5 KB)


val: Scanning /Users/admin/Car-detection-serving-model/open_images_project/dataset/labels/val.cache... 400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 400/400 [00:00<?, ?it/s]

Plotting labels to /Users/admin/runs/detect/train_large_new_final/labels.jpg... 


...

      10/10         0G     0.9402     0.8262      1.178         39        640: 100%|██████████| 225/225 [31:13<00:00,  8.33s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:56<00:00,  4.36s/it]

                   all        400       1203      0.767       0.66      0.739      0.527



10 epochs completed in 5.345 hours.
Optimizer stripped from /Users/admin/runs/detect/train_large_new_final/weights/last.pt, 5.5MB
Optimizer stripped from /Users/admin/runs/detect/train_large_new_final/weights/best.pt, 5.5MB

Validating /Users/admin/runs/detect/train_large_new_final/weights/best.pt...
Ultralytics 8.3.115 🚀 Python-3.11.4 torch-2.2.2 CPU (Intel Core(TM) i7-8559U 2.70GHz)
YOLO11n summary (fused): 100 layers, 2,582,932 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:49<00:00,  3.81s/it]


                   all        400       1203      0.767      0.666      0.736      0.527
                   Car        167        592      0.667      0.537      0.588       0.39
                  Taxi        104        298      0.779      0.641      0.755      0.486
                 Truck        103        136      0.847      0.728      0.816      0.614
                   Bus        101        177      0.776      0.757      0.782       0.62
Speed: 3.0ms preprocess, 97.1ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to /Users/admin/runs/detect/train_large_new_final


In [16]:
results = model.predict(
    source="/Users/admin/Car-detection-serving-model/open_images_project/dataset/images/val",
    save=True
)


image 1/400 /Users/admin/Car-detection-serving-model/open_images_project/dataset/images/val/0044141d3732acc5.jpg: 640x640 6 Cars, 337.7ms
image 2/400 /Users/admin/Car-detection-serving-model/open_images_project/dataset/images/val/006f44b210255845.jpg: 448x640 2 Buss, 228.5ms
image 3/400 /Users/admin/Car-detection-serving-model/open_images_project/dataset/images/val/007cdb3928bcfdb2.jpg: 448x640 2 Cars, 1 Taxi, 391.1ms
image 4/400 /Users/admin/Car-detection-serving-model/open_images_project/dataset/images/val/007cdfad5b85d3e7.jpg: 640x448 1 Truck, 1 Bus, 233.3ms
image 5/400 /Users/admin/Car-detection-serving-model/open_images_project/dataset/images/val/009b067a652d7d79.jpg: 608x640 1 Bus, 180.5ms
image 6/400 /Users/admin/Car-detection-serving-model/open_images_project/dataset/images/val/00ef407e5b2ef618.jpg: 448x640 1 Bus, 132.7ms
image 7/400 /Users/admin/Car-detection-serving-model/open_images_project/dataset/images/val/01133647412c00b0.jpg: 480x640 1 Car, 172.5ms
image 8/400 /Users/a

In [82]:
import mlflow
import os
import pandas as pd
from ultralytics import YOLO
from mlflow.tracking import MlflowClient
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, TensorSpec
import numpy as np

# Đường dẫn
train_dir = "/Users/admin/runs/detect/train_large_new_final"
data_yaml = "/Users/admin/Car-detection-serving-model/open_images_project/dataset/data.yaml"
model_path = "/Users/admin/runs/detect/train_large_new_final/weights/best.pt"
results_path = "/Users/admin/runs/detect/train_large_new_final/results.csv"

# Thiết lập tracking URI và experiment
mlflow.set_tracking_uri("sqlite:///mlruns.db")
mlflow.set_experiment("YOLOv11_Experiments")

# Kiểm tra file results.csv tồn tại
if not os.path.exists(results_path):
    raise FileNotFoundError(f"File {results_path} does not exist. Please run the training step first.")

# Đọc metrics từ results.csv
results_df = pd.read_csv(results_path)
metrics_from_csv = {
    "mAP50": results_df["metrics/mAP50(B)"].iloc[-1],
    "mAP50-95": results_df["metrics/mAP50-95(B)"].iloc[-1],
    "train_box_loss": results_df["train/box_loss"].iloc[-1],
    "val_box_loss": results_df["val/box_loss"].iloc[-1]
}

# Bắt đầu một run duy nhất để log tất cả thông tin
with mlflow.start_run(run_name="YOLOv11_Training_Final") as run:
    # Log tham số huấn luyện
    params = {
        "epochs": 60,
        "imgsz": 640,
        "batch": 16,
        "optimizer": "AdamW",
        "lr0": 0.0005
    }
    mlflow.log_params(params)

    # Log artifacts
    mlflow.log_artifact(data_yaml)
    mlflow.log_artifact(train_dir)
    mlflow.log_artifact(model_path)
    mlflow.log_artifact(results_path)
    mlflow.log_artifact("/Users/admin/runs/detect/train_large_new_final/results.png", "training_plots")
    print("Đã log artifacts vào MLflow")

    # Log metrics từ results.csv
    mlflow.log_metrics(metrics_from_csv)
    print("Đã log metrics vào MLflow")

    # Định nghĩa signature thủ công (nếu cần log mô hình)
    input_schema = Schema([TensorSpec(np.dtype(np.float32), (-1, 3, 640, 640))])
    output_schema = Schema([TensorSpec(np.dtype(np.float32), (-1, -1))])
    signature = ModelSignature(inputs=input_schema, outputs=output_schema)

    # Log mô hình bằng cách log file best.pt (thay vì mlflow.pytorch.log_model)
    mlflow.log_artifact(model_path)

    # Đăng ký mô hình
    model_uri = f"runs:/{run.info.run_id}/model"
    model_name = "YOLOv11_Model"
    result = mlflow.register_model(model_uri, model_name)
    print(f"Mô hình đã được đăng ký với version {result.version}")

    # Thêm tags
    client = MlflowClient()
    tags = {
        "model_type": "YOLOv11",
        "task": "vehicle_detection",
        "imgsz": "640",
        "epochs": "60",
        "optimizer": "AdamW"
    }
    for key, value in tags.items():
        client.set_model_version_tag(model_name, result.version, key, value)
    client.set_registered_model_tag(model_name, "description", "YOLOv11 model for vehicle detection")

    # Chuyển stage Production nếu mAP50 >= 0.7
    mAP50 = metrics_from_csv["mAP50"]
    if mAP50 >= 0.7:
        client.transition_model_version_stage(
            name=model_name,
            version=result.version,
            stage="Production"
        )
        print(f"Model version {result.version} transitioned to Production stage")

# In kết quả metrics cuối cùng để kiểm tra
print("Metrics từ results.csv:")
print(metrics_from_csv)

Đã log artifacts vào MLflow
Đã log metrics vào MLflow
Mô hình đã được đăng ký với version 7


Registered model 'YOLOv11_Model' already exists. Creating a new version of this model...
Created version '7' of model 'YOLOv11_Model'.


Model version 7 transitioned to Production stage
Metrics từ results.csv:
{'mAP50': 0.73854, 'mAP50-95': 0.52683, 'train_box_loss': 0.94022, 'val_box_loss': 1.08893}


In [83]:
# Chạy MLflow UI trên port 5001 với backend store mới
!mlflow ui --backend-store-uri file:///Users/admin/Documents/Car-detection-serving-model/open_images_project/mlruns_new --port 5001

[2025-05-03 22:47:46 +0700] [57051] [INFO] Starting gunicorn 23.0.0
[2025-05-03 22:47:46 +0700] [57051] [INFO] Listening at: http://127.0.0.1:5001 (57051)
[2025-05-03 22:47:46 +0700] [57051] [INFO] Using worker: sync
[2025-05-03 22:47:46 +0700] [57052] [INFO] Booting worker with pid: 57052
[2025-05-03 22:47:46 +0700] [57053] [INFO] Booting worker with pid: 57053
[2025-05-03 22:47:46 +0700] [57054] [INFO] Booting worker with pid: 57054
[2025-05-03 22:47:46 +0700] [57055] [INFO] Booting worker with pid: 57055
^C

Aborted!
[2025-05-03 22:48:10 +0700] [57052] [INFO] Worker exiting (pid: 57052)
[2025-05-03 22:48:10 +0700] [57055] [INFO] Worker exiting (pid: 57055)
[2025-05-03 22:48:10 +0700] [57053] [INFO] Worker exiting (pid: 57053)
[2025-05-03 22:48:10 +0700] [57054] [INFO] Worker exiting (pid: 57054)
